In [ ]:
import os
print(os.getcwd())
! mkdir -p prod && mkdir -p log && which python

## Example how to find all unit tests

In [ ]:
mk_dir = False
test_case_directories = set()
for root, dirnames, files in os.walk(os.getcwd()):
    for filename in files:  # loop through files in the current directory
        file_name = os.path.join(root, filename)
        if file_name.startswith(f"{os.getcwd()}/data/"):
            test_case_name = file_name[0:file_name.rfind("/")].replace(f"{os.getcwd()}/data/", "")
            if not test_case_name.startswith("/home"):
                test_case_directories.add(test_case_name)
        del file_name
for test_case in test_case_directories:
    # print(test_case)
    for subdir in ["prod", "log", "reference"]:
        fpath = f"{os.getcwd()}/{subdir}/{test_case}"
        if not(os.path.exists(fpath) and os.path.isdir(fpath)):
            if mk_dir:
                os.mkdir(fpath)
        del fpath

## Generate list of all tests for code launch.json

In [ ]:
for test_case in sorted(test_case_directories):
    file_list = os.listdir(f"{os.getcwd()}/data/{test_case}")
    # for file_name in file_list:
    #    print(f'''                     //"tests/data/{test_case}/{file_name}",''')
    print(f'''                     //"tests/prod/{test_case}/output.nxs"]''')    

## Run tests

In [ ]:
run_parser = True
verbose = True
whitelist = []  # "default"]
blacklist = []
n_tests = 0
for test_case in sorted(test_case_directories):
    if len(whitelist) > 0:
        if test_case not in whitelist:
            continue
    if len(blacklist) > 0:
        if test_case in blacklist:
            continue
    file_list = os.listdir(f"{os.getcwd()}/data/{test_case}")
    print(f"- [ ] {test_case}")  # , {len(file_list)}")
    continue
    
    curr_dir = f"{os.getcwd()}/data/{test_case}"
    out = f"{os.getcwd()}/prod/{test_case}/output.nxs"  # data.{parser_type}.{entry}.nxs"
    stdout = f"{os.getcwd()}/log/{test_case}/stdout.{test_case}.txt"
    stderr = f"{os.getcwd()}/log/{test_case}/stderr.{test_case}.txt"
    if len(file_list) == 1:
        in_one = f"{curr_dir}/{file_list[0]}"
        if run_parser:
            ! dataconverter $in_one --reader em --nxdl NXem --output $out 1>$stdout 2>$stderr
            n_tests += 1
        del in_one
    elif len(file_list) == 2:
        in_one = f"{curr_dir}/{file_list[0]}"
        in_two = f"{curr_dir}/{file_list[1]}"
        if run_parser:
            ! dataconverter $in_one $in_two --reader em --nxdl NXem --output $out 1>$stdout 2>$stderr
            n_tests += 1
        del in_one, in_two
    elif len(file_list) == 3:
        in_one = f"{curr_dir}/{file_list[0]}"
        in_two = f"{curr_dir}/{file_list[1]}"
        in_three = f"{curr_dir}/{file_list[2]}"
        if run_parser:
            ! dataconverter $in_one $in_two $in_three --reader em --nxdl NXem --output $out 1>$stdout 2>$stderr
        del in_one, in_two, in_three
    del file_list, curr_dir, out, stdout, stderr
print(f"Ran {n_tests} tests")

## Example how to generate a reference artifact from a NeXus file for unit tests

In [ ]:
import os
from pynxtools_em.parsers.hfive_base import HdfFiveBaseParser, NXEM_VOLATILE_METADATA

fpath = f"{os.getcwd()}/data/eln_second/output.nxs"
hfive_parser = HdfFiveBaseParser(
    file_path=fpath, hashing=True, verbose=False
)
hfive_parser.get_content()
hfive_parser.store_hashes(
    blacklist=NXEM_VOLATILE_METADATA, file_path=f"{os.getcwd()}/reference/eln_second/output.nxs.sha256.ref.yaml"
)

***

## Define tests

In [ ]:
tests = {
    "eln": [("eln_data.yaml", "em.oasis.specific.yaml")],
    "default": [("output.nxs_eln_data.yaml", "InGaN_nanowires_spectra.edaxh5")], 
    "nxs_nion": ["2022-02-18_Metadata_Kuehbach.zip"],
    "rsciio_velox": ["258.fbbd9cdfcf8be1b1d6e056cdf4186a0505df9188bbf52ebe36e74e271a8c972b.emd", "265.2dccb74e742d9807d736689ee1bdeb149aabbbccbecab89ab0d5855678a2b338.emd"],
    "image_tfs": ["ALN_baoh_021.tif", "ETD_image.tif", "T3_image.tif", "NavCam_normal_vis_light_ccd.tif", "0c8nA_3deg_003_AplusB_test.tif"],
    "rsciio_gatan": ["262.b97850f6c6b100740813c34d20eda294ef66a6130daa42a6cf975c4572ff599c.dm3", "265.6a232b951fe99bb06c481e6863b5b590df194b0766c915218586c81077417e8a.dm4"],

    "image_hitachi": [("360.tif", "360.txt")],
    "image_jeol": [("20240227_A1_2m_0_FA3_1.tif", "20240227_A1_2m_0_FA3_1.txt")],
    "image_zeiss": ["SE2.tif"],
    "image_tescan": [("Ti3C2VacDriedFilm19.tif", "Ti3C2VacDriedFilm19.hdr"), "CZ04-2_102_Pic_2.tif"],
    "image_point_electronic": ["Defekt1.tif"],
    "image_protochips": ["ReductionOfFeOxSmall.zip"],
    "image_fei_legacy": ["BF_02_40kx.tif", "SEM_Image_-_SliceImage_-_109.tif"],

    "hfive_oxford": ["57764_CR_ZG_380.h5oina", "EBSDCleanedMapData7.h5oina", "173_0057.h5oina", "173_0056.h5oina"],  
    "hfive_apex": ["InGaN_nanowires_map.edaxh5", "InGaN_nanowires_linescan.edaxh5", "InGaN_nanowires_spectra.edaxh5"], # "2023-08-16_Ni_NFDI.edaxh5", "AlGaO.h5", "VInP_108_L2.h5"],
    "hfive_bruker": ["066_0025.h5"],
    "hfive_edax": ["001_0050.h5", "229_2097.oh5"],

    "image_diffraction_pattern": ["original_data.zip"],
    "conventions": ["em.conventions.yaml"],
    #"hfive_dreamthreed_legacy": ["067_0003.dream3d", "SmallIN100_Final.dream3d", "244_0014.dream3d"],
}